<a href="https://colab.research.google.com/github/leesolhahaha/study/blob/main/KoBERT%EB%A5%BC_%ED%99%9C%EC%9A%A9%ED%95%9C_%EA%B0%90%EC%A0%95%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## 오류 발생(미해결)

In [ ]:
## 설치 후 런타임 다시실행
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.5 MB/s eta 0:00:00


In [ ]:
# Colab 환경 설정
# requirements : https://github.com/SKTBrain/KoBERT/blob/master/kobert_hf/requirements.txt
!pip install gluonnlp pandas tqdm
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=645678 sha256=a066d09ebfa5d66f7188ac181cbdad0abe123bc3321341558835c6f1d035ba39
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 24.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 5.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadat

In [ ]:
# https://github.com/SKTBrain/KoBERT 의 파일들을 Colab으로 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install gluonnlp==0.9.1

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-_dzcm6l6/kobert-tokenizer_0d186ce60c264aeaa50da8d602c5dae2
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-_dzcm6l6/kobert-tokenizer_0d186ce60c264aeaa50da8d602c5dae2
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=bf047d9f8e84c0f1ef2bdfdc8bc2c0be13b8b25a823d7803ce68904d3a4a1060
  Stored in directory: /tmp/pip-ephem-wheel-cache-mmrc5c5l/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp310-cp310-linux_x86_64.whl size=530292 sha256=1e6444c

In [ ]:
!pip install kobert_tokenizer

In [ ]:
# koBERT
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
import transformers
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

In [ ]:
# Transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# Setting Library
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd


In [ ]:
device = torch.device("cuda:0")


In [ ]:
# BERT 모델, Vocabulary 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# [AI Hub] 감정 분류를 위한 대화 음성 데이터셋
data = pd.read_csv("/content/drive/MyDrive/데이터셋/감정대화/5차년도_2차.csv", encoding='cp949')


In [ ]:
data['상황'].unique()

array(['happiness', 'neutral', 'sadness', 'angry', 'surprise', 'disgust',
       'fear'], dtype=object)

In [ ]:
# 7개의 감정 class → 숫자 0: 부정 | 1: 평범
data.loc[(data['상황'] == "fear"), '상황'] = 0  # fear(두려움) → 0
data.loc[(data['상황'] == "surprise"), '상황'] = 1  # surprise(놀람) → 1
data.loc[(data['상황'] == "angry"), '상황'] = 0  # angry(화남) → 0
data.loc[(data['상황'] == "sadness"), '상황'] = 0  # sadness(슬픔) → 0
data.loc[(data['상황'] == "neutral"), '상황'] = 1  # neutral(중립) → 1
data.loc[(data['상황'] == "happiness"), '상황'] = 1  # happiness(기쁨) → 1
data.loc[(data['상황'] == "disgust"), '상황'] = 0  # disgust(싫음) → 0

In [ ]:
data['상황'].unique()

array([1, 0], dtype=object)

In [ ]:
# [발화문, 상황] data_list 생성
data_list = []
for ques, label in zip (data['발화문'], data['상황']):
  data = []
  data.append(ques)
  data.append(str(label))

  data_list.append(data)

In [ ]:
for i in data_list:
    if i[1] == '1':
        i[1] = 1
    elif i[1] == '0':
        i[1] = 0

In [ ]:
print(data)
print(data_list[:10])


['걱정해줘서 고마워.', '0']
[['헐! 나 이벤트에 당첨 됐어.', '1'], ['내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.', '1'], ['한 명 뽑는 거였는데, 그게 바로 내가 된 거야.', '1'], ['당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. 그래서 못 산 향수야.', '1'], ['에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.', '1'], ['난 부페 형식의 음식들도 정말 좋아해. 그 것도 좀 알려 줘.', '1'], ['응. 완전히 끝난 거야. 한 달 동안 주말에 쉬지도 못하고 일만 했거든.', '1'], ['신나는 음악 듣는 것도 좋고, 어디 여행 가고 싶고 이 것 저 것 다 해보고 싶어.', '1'], ['친구들도 내 연락 기다리고 있을 텐데 내가 까먹고 있었네?', '1'], ['그래. 일단은 친구들부터 만나서 여행 계획에 대해서 얘기 좀 해봐야 되겠어.', '1']]


In [ ]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 32)


In [ ]:
print(len(dataset_train), len(dataset_test))

15499 3875


In [ ]:

tok = nlp.data.BERTSPTokenizer(tokenizer, vocab)

In [ ]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
# 출처 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab = vocab, pad = pad, pair = pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))


In [ ]:
# parameter 값 출처 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5


In [ ]:
for data_point in dataset_train[:5]:  # 처음 5개 데이터만 출력
    print(data_point)

['조금 더 기다려볼까?', '0']
['해피랑 산책하다가 목줄이 끊겨버렸어. 그래서 붙잡느라고 혼이 났어.', '1']
['오랜만에 가족들이랑 동해 바다에 가서 놀다 오기로 했어. 근데 비가 와서 망했어.', '0']
['어 그것도 너무 좋을 것 같아.', '1']
['나 요즘 주식 시작했어.', '0']


In [ ]:
# Tokenizer 객체 확인
print(tokenizer.vocab_size)  # 어휘 크기
print(tokenizer.pad_token)    # 패딩 토큰
print(tokenizer.tokenize("예시 문장입니다."))  # 토큰화 예시

8002
[PAD]
['▁예', '시', '▁문', '장', '입니다', '.']


In [ ]:
# Vocab 객체 확인
print(vocab.idx_to_token[0])  # 인덱스 0에 해당하는 토큰
print(vocab.token_to_idx['[PAD]'])  # 토큰 '[PAD]'에 해당하는 인덱스

[UNK]
1


In [ ]:
nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [ ]:
print(dataset_train[:10])
print(dataset_test[:10])

[['조금 더 기다려볼까?', '0'], ['해피랑 산책하다가 목줄이 끊겨버렸어. 그래서 붙잡느라고 혼이 났어.', '1'], ['오랜만에 가족들이랑 동해 바다에 가서 놀다 오기로 했어. 근데 비가 와서 망했어.', '0'], ['어 그것도 너무 좋을 것 같아.', '1'], ['나 요즘 주식 시작했어.', '0'], ['아니 원래는 바닷가 가 가지고 그 옆에 팬션에서 놀면서 경치 구경도 좀 하고 해산물도 먹으려고 그랬지.', '0'], ['그렇잖아도 내일 다 약속 했어. 만나기로.', '1'], ['그래. 기분전환은 해야되겠어. 너무 처지니까 안되겠어.', '0'], ['뭐 그럭저럭.', '1'], ['알았어. 지금 당장 사진을 첨부해서 전화해볼게!', '0']]
[['그러게 그것도 나쁘지 않겠다. 찾아봐야겠어.', '0'], ['너의 말도 맞는 것 같아. 욕한다고 기분이 나아지진 않더라. 나는 신나는 음악을 들으면 기분이 좋아져.', '0'], ['환기때문인지 미세먼지 때문인지. 아 곰팡이 때문에 짜증나 죽겠어.', '0'], ['내가 10km 마라톤 대회에 출전했는데 이전의 내 기록을 훨씬 갱신했어.', '1'], ['응! 우리 동아리 사람들이랑 같이 뛰어.', '1'], ['음식점과 유동 인구가 많은 강남역이야.', '1'], ['와. 짜장면에서 벌레 나왔다.', '0'], ['잘 모르고 그냥 친구가 추천해주는거 시작했어.', '0'], ['좋아. 아무튼 너무 신난다.', '1'], ['해피가 달려가서 사람들이 너무 놀랐어.', '1']]


In [ ]:
print(type(dataset_train))
print(type(dataset_test))

<class 'list'>
<class 'list'>


In [ ]:
print("Tokenization Model (tok):", tok)
print("Vocabulary (vocab):", vocab)

Tokenization Model (tok): <gluonnlp.data.transforms.BERTSPTokenizer object at 0x7cb911265840>
Vocabulary (vocab): Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")


In [ ]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
# BERTDataset 생성
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

TypeError: ignored

In [ ]:
https://bbarry-lee.github.io/ai-tech/KoBERT%EB%A5%BC-%ED%99%9C%EC%9A%A9%ED%95%9C-%EA%B0%90%EC%A0%95%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EA%B5%AC%ED%98%84.html